<a href="https://colab.research.google.com/github/TrangleUET/ReID/blob/main/ReID.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# cd to your preferred directory and clone this repo
!git clone https://github.com/KaiyangZhou/deep-person-reid.git

# create environment
%cd deep-person-reid/

# install dependencies
# make sure `which python` and `which pip` point to the correct path
!pip install -r requirements.txt

# install torchreid (don't need to re-build it if you modify the source code)
!python setup.py develop

Cloning into 'deep-person-reid'...
remote: Enumerating objects: 9809, done.
remote: Counting objects: 100% (806/806), done.
remote: Compressing objects: 100% (95/95), done.
remote: Total 9809 (delta 741), reused 711 (delta 711), pack-reused 9003
Receiving objects: 100% (9809/9809), 9.50 MiB | 21.47 MiB/s, done.
Resolving deltas: 100% (7284/7284), done.
/content/deep-person-reid/deep-person-reid
/content/deep-person-reid/deep-person-reid/torchreid/metrics/rank.py:12: UserWarning: Cython evaluation (very fast so highly recommended) is unavailable, now use python evaluation.
  'Cython evaluation (very fast so highly recommended) is '
Compiling torchreid/metrics/rank_cylib/rank_cy.pyx because it changed.
[1/1] Cythonizing torchreid/metrics/rank_cylib/rank_cy.pyx
/usr/local/lib/python3.7/dist-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /content/deep-person-reid/deep-person-r

In [6]:
import torchreid

In [ ]:
datamanager = torchreid.data.ImageDataManager(
    root='reid-data',
    sources='market1501',
    targets='market1501',
    height=256,
    width=128,
    batch_size_train=32,
    batch_size_test=100,
    transforms=['random_flip', 'random_crop']
)

Building train transforms ...
+ resize to 256x128
+ random flip
+ random crop (enlarge to 288x144 and crop 256x128)
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
Building test transforms ...
+ resize to 256x128
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
=> Loading train (source) dataset
Creating directory "/content/deep-person-reid/reid-data/market1501"
* url="http://188.138.127.15:81/Datasets/Market-1501-v15.09.15.zip"
* destination="/content/deep-person-reid/reid-data/market1501/Market-1501-v15.09.15.zip"
...100%, 145 MB, 3332 KB/s, 44 seconds passed
Extracting "/content/deep-person-reid/reid-data/market1501/Market-1501-v15.09.15.zip"
Market1501 dataset is ready
=> Loaded Market1501
  ----------------------------------------
  subset   | # ids | # images | # cameras
  ----------------------------------------
  train    |   751 |    12936 |         6
  query    | 

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


=> Loaded Market1501
  ----------------------------------------
  subset   | # ids | # images | # cameras
  ----------------------------------------
  train    |   751 |    12936 |         6
  query    |   750 |     3368 |         6
  gallery  |   751 |    15913 |         6
  ----------------------------------------


  **************** Summary ****************
  source            : ['market1501']
  # source datasets : 1
  # source ids      : 751
  # source images   : 12936
  # source cameras  : 6
  target            : ['market1501']
  *****************************************




In [ ]:
model = torchreid.models.build_model(
    name='resnet50',
    num_classes=datamanager.num_train_pids,
    loss='softmax',
    pretrained=True
)

model = model.cuda()

optimizer = torchreid.optim.build_optimizer(
    model,
    optim='adam',
    lr=0.0003
)

scheduler = torchreid.optim.build_lr_scheduler(
    optimizer,
    lr_scheduler='single_step',
    stepsize=20
)

engine = torchreid.engine.ImageSoftmaxEngine(
    datamanager,
    model,
    optimizer=optimizer,
    scheduler=scheduler,
    label_smooth=True
)

engine.run(
    save_dir='log/resnet50',
    max_epoch=10,
    eval_freq=10,
    print_freq=10,
    test_only=False
)

=> Start training


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch: [1/10][10/404]	time 0.471 (0.518)	data 0.001 (0.041)	eta 0:34:46	loss 6.9096 (6.7987)	acc 3.1250 (0.3125)	lr 0.000300
epoch: [1/10][20/404]	time 0.472 (0.494)	data 0.006 (0.021)	eta 0:33:06	loss 6.9636 (6.8996)	acc 6.2500 (0.7812)	lr 0.000300
epoch: [1/10][30/404]	time 0.474 (0.486)	data 0.001 (0.014)	eta 0:32:28	loss 6.6295 (6.8206)	acc 0.0000 (0.9375)	lr 0.000300
epoch: [1/10][40/404]	time 0.474 (0.482)	data 0.002 (0.011)	eta 0:32:08	loss 6.4807 (6.7794)	acc 3.1250 (0.8594)	lr 0.000300
epoch: [1/10][50/404]	time 0.468 (0.480)	data 0.000 (0.009)	eta 0:31:54	loss 6.6173 (6.7570)	acc 0.0000 (0.8125)	lr 0.000300
epoch: [1/10][60/404]	time 0.469 (0.478)	data 0.000 (0.008)	eta 0:31:42	loss 6.5728 (6.7364)	acc 0.0000 (0.7292)	lr 0.000300
epoch: [1/10][70/404]	time 0.469 (0.477)	data 0.001 (0.007)	eta 0:31:32	loss 6.6631 (6.7209)	acc 0.0000 (0.6696)	lr 0.000300
epoch: [1/10][80/404]	time 0.473 (0.476)	data 0.001 (0.006)	eta 0:31:24	loss 6.4996 (6.7056)	acc 0.0000 (0.5859)	lr 0.000300


In [ ]:
model = torchreid.models.build_model(
    name='osnet_x1_0',
    num_classes=datamanager.num_train_pids,
    loss='softmax',
    pretrained=True
)

model = model.cuda()

optimizer = torchreid.optim.build_optimizer(
    model,
    optim='adam',
    lr=0.0003
)

scheduler = torchreid.optim.build_lr_scheduler(
    optimizer,
    lr_scheduler='single_step',
    stepsize=20
)

engine = torchreid.engine.ImageSoftmaxEngine(
    datamanager,
    model,
    optimizer=optimizer,
    scheduler=scheduler,
    label_smooth=True
)

engine.run(
    save_dir='log/osnet_x1_0',
    max_epoch=10,
    eval_freq=10,
    print_freq=10,
    test_only=False
)

Downloading...
From: https://drive.google.com/uc?id=1LaG1EJpHrxdAxKnSCJ_i0u-nbxSAeiFY
To: /root/.cache/torch/checkpoints/osnet_x1_0_imagenet.pth
100%|██████████| 10.9M/10.9M [00:00<00:00, 190MB/s]
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Successfully loaded imagenet pretrained weights from "/root/.cache/torch/checkpoints/osnet_x1_0_imagenet.pth"
** The following layers are discarded due to unmatched keys or layer size: ['classifier.weight', 'classifier.bias']
=> Start training
epoch: [1/10][10/404]	time 0.504 (0.582)	data 0.002 (0.070)	eta 0:39:05	loss 6.6352 (6.6483)	acc 0.0000 (0.0000)	lr 0.000300
epoch: [1/10][20/404]	time 0.503 (0.543)	data 0.000 (0.035)	eta 0:36:22	loss 6.7139 (6.6441)	acc 3.1250 (0.1562)	lr 0.000300
epoch: [1/10][30/404]	time 0.503 (0.529)	data 0.000 (0.024)	eta 0:35:23	loss 6.5112 (6.6156)	acc 6.2500 (0.5208)	lr 0.000300
epoch: [1/10][40/404]	time 0.505 (0.523)	data 0.000 (0.018)	eta 0:34:52	loss 6.4442 (6.5947)	acc 3.1250 (1.1719)	lr 0.000300
epoch: [1/10][50/404]	time 0.507 (0.519)	data 0.000 (0.015)	eta 0:34:31	loss 6.5004 (6.5731)	acc 3.1250 (1.5000)	lr 0.000300
epoch: [1/10][60/404]	time 0.499 (0.517)	data 0.001 (0.012)	eta 0:34:15	loss 6.6807 (6.5451)	acc 0.0000 (1.4583)	lr 0.000300
epoch:

In [ ]:
model = torchreid.models.build_model(
    name='osnet_ain_x1_0',
    num_classes=datamanager.num_train_pids,
    loss='softmax',
    pretrained=True
)

model = model.cuda()

optimizer = torchreid.optim.build_optimizer(
    model,
    optim='adam',
    lr=0.0003
)

scheduler = torchreid.optim.build_lr_scheduler(
    optimizer,
    lr_scheduler='single_step',
    stepsize=20
)

engine = torchreid.engine.ImageSoftmaxEngine(
    datamanager,
    model,
    optimizer=optimizer,
    scheduler=scheduler,
    label_smooth=True
)

engine.run(
    save_dir='log/osnet_x1_0',
    max_epoch=10,
    eval_freq=10,
    print_freq=10,
    test_only=False
)

Downloading...
From: https://drive.google.com/uc?id=1-CaioD9NaqbHK_kzSMW8VE4_3KcsRjEo
To: /root/.cache/torch/checkpoints/osnet_ain_x1_0_imagenet.pth
100%|██████████| 10.9M/10.9M [00:00<00:00, 174MB/s]


Successfully loaded imagenet pretrained weights from "/root/.cache/torch/checkpoints/osnet_ain_x1_0_imagenet.pth"
** The following layers are discarded due to unmatched keys or layer size: ['classifier.weight', 'classifier.bias']
=> Start training


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch: [1/10][10/404]	time 0.515 (0.589)	data 0.000 (0.067)	eta 0:39:34	loss 6.6668 (6.6482)	acc 0.0000 (0.6250)	lr 0.000300
epoch: [1/10][20/404]	time 0.510 (0.551)	data 0.001 (0.034)	eta 0:36:54	loss 6.4260 (6.6190)	acc 6.2500 (0.9375)	lr 0.000300
epoch: [1/10][30/404]	time 0.513 (0.538)	data 0.002 (0.023)	eta 0:35:56	loss 6.6159 (6.5869)	acc 3.1250 (1.2500)	lr 0.000300
epoch: [1/10][40/404]	time 0.507 (0.530)	data 0.001 (0.017)	eta 0:35:21	loss 6.4172 (6.5654)	acc 3.1250 (1.8750)	lr 0.000300
epoch: [1/10][50/404]	time 0.508 (0.526)	data 0.000 (0.014)	eta 0:34:58	loss 6.0984 (6.5343)	acc 0.0000 (1.9375)	lr 0.000300
epoch: [1/10][60/404]	time 0.514 (0.523)	data 0.000 (0.012)	eta 0:34:41	loss 6.0830 (6.5121)	acc 9.3750 (2.0833)	lr 0.000300
epoch: [1/10][70/404]	time 0.508 (0.521)	data 0.001 (0.010)	eta 0:34:29	loss 5.8559 (6.4717)	acc 12.5000 (2.6786)	lr 0.000300
epoch: [1/10][80/404]	time 0.505 (0.519)	data 0.000 (0.009)	eta 0:34:16	loss 6.1998 (6.4322)	acc 0.0000 (3.2031)	lr 0.000300